In [1]:
import sqlite3

# Connect to the SQLite database (it will create one if it doesn't exist)
conn = sqlite3.connect("banking_system.db")
cursor = conn.cursor()

# Create the users table
cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    account_number TEXT UNIQUE NOT NULL,
    dob TEXT,
    city TEXT,
    password TEXT NOT NULL,
    initial_balance REAL NOT NULL,
    contact_number TEXT,
    email_id TEXT,
    address TEXT
)
''')

# Create the login table
cursor.execute('''
CREATE TABLE IF NOT EXISTS login (
    user_id INTEGER PRIMARY KEY,
    account_number TEXT UNIQUE NOT NULL,
    password TEXT NOT NULL,
    FOREIGN KEY(user_id) REFERENCES users(user_id)
)
''')

# Create the transaction table with a renamed table name
cursor.execute('''
CREATE TABLE IF NOT EXISTS transactions (
    transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_id INTEGER NOT NULL,
    type TEXT NOT NULL,
    amount REAL NOT NULL,
    date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY(user_id) REFERENCES users(user_id)
)
''')


# Commit the changes and close the connection
conn.commit()
conn.close()


In [2]:
import tkinter as tk
from tkinter import messagebox, ttk
import random
import string
import re
import sqlite3
from tkcalendar import Calendar

# List of cities in India (you can expand this as per your requirement)
cities_in_india = [
    "Delhi", "Mumbai", "Bangalore", "Chennai", "Hyderabad", "Kolkata", "Ahmedabad", "Pune", 
    "Jaipur", "Surat", "Lucknow", "Kanpur", "Nagpur", "Indore", "Patna", "Vadodara", "Ghaziabad",
    "Ludhiana", "Agra", "Nashik", "Faridabad", "Meerut", "Kochi", "Coimbatore", "Visakhapatnam", 
    "Bhopal", "Ranchi", "Chandigarh", "Mysore", "Rajkot", "Dhanbad", "Jabalpur", "Vijayawada"
]

# List of all States and Union Territories of India
states_in_india = [
    "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chhattisgarh", "Goa", "Gujarat", 
    "Haryana", "Himachal Pradesh", "Jharkhand", "Karnataka", "Kerala", "Madhya Pradesh", "Maharashtra", 
    "Manipur", "Meghalaya", "Mizoram", "Nagaland", "Odisha", "Punjab", "Rajasthan", "Sikkim", 
    "Tamil Nadu", "Telangana", "Tripura", "Uttar Pradesh", "Uttarakhand", "West Bengal", "Andaman and Nicobar Islands", 
    "Chandigarh", "Dadra and Nagar Haveli and Daman and Diu", "Lakshadweep", "Delhi", "Puducherry", 
    "Ladakh", "Lakshadweep", "Ladakh", "Jammu and Kashmir", "Nagaland"
]

class BankingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Banking System")
        self.root.geometry("600x600")
        self.root.config(bg="#F5F5DC")

        # Create form elements
        self.create_user_form()

    def create_user_form(self):
        # Frame for user input
        user_form_frame = tk.Frame(self.root, bg="#F5F5DC")
        user_form_frame.pack(pady=20)

        # Name field
        self.name_label = tk.Label(user_form_frame, text="Name:", bg="#F5F5DC", fg="#6F4F37", font=("Arial", 12))
        self.name_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        self.name_entry = tk.Entry(user_form_frame, font=("Arial", 12))
        self.name_entry.grid(row=0, column=1)

        # Date of birth button (using calendar popup code)
        self.dob_button = tk.Button(user_form_frame, text="Date of Birth", command=self.show_calendar, font=("Arial", 12), bg="#6F4F37", fg="white")
        self.dob_button.grid(row=1, column=0, padx=10, pady=10)

        # City dropdown with autocomplete
        self.city_label = tk.Label(user_form_frame, text="City:", bg="#F5F5DC", fg="#6F4F37", font=("Arial", 12))
        self.city_label.grid(row=2, column=0, padx=10, pady=10, sticky="w")
        
        self.city_combobox = ttk.Combobox(user_form_frame, values=cities_in_india, font=("Arial", 12), width=20)
        self.city_combobox.grid(row=2, column=1, padx=10, pady=10)
        self.city_combobox.set("")

        # State dropdown with all Indian states
        self.state_label = tk.Label(user_form_frame, text="State:", bg="#F5F5DC", fg="#6F4F37", font=("Arial", 12))
        self.state_label.grid(row=3, column=0, padx=10, pady=10, sticky="w")
        self.state_combobox = ttk.Combobox(user_form_frame, values=states_in_india, font=("Arial", 12), width=20)
        self.state_combobox.grid(row=3, column=1, padx=10, pady=10)
        self.state_combobox.set("")

        # Password field
        self.password_label = tk.Label(user_form_frame, text="Password:", bg="#F5F5DC", fg="#6F4F37", font=("Arial", 12))
        self.password_label.grid(row=4, column=0, padx=10, pady=10, sticky="w")
        self.password_entry = tk.Entry(user_form_frame, show="*", font=("Arial", 12))
        self.password_entry.grid(row=4, column=1)

        # Submit button
        self.submit_button = tk.Button(user_form_frame, text="Add User", font=("Arial", 14), bg="#6F4F37", fg="white", command=self.add_user)
        self.submit_button.grid(row=5, column=0, columnspan=2, pady=20)

    def add_user(self):
        # Get user data
        name = self.name_entry.get()
        dob = self.dob_button["text"]  # Assuming you set the selected date here
        city = self.city_combobox.get()
        state = self.state_combobox.get()
        password = self.password_entry.get()

        # Validation for required fields
        if not name or not dob or not city or not state or not password:
            messagebox.showerror("Error", "All fields are required.")
            return

        # Here you can add code to store the user in a database
        messagebox.showinfo("Success", "User added successfully")



    def show_calendar(self):
        if not self.calendar_window or not self.calendar_window.winfo_exists():  # Check if the window exists
            # Create a new calendar window if it doesn't exist
            self.calendar_window = tk.Toplevel(self.root)
            self.calendar_window.title("Select Date of Birth")
            self.calendar_window.geometry("300x300")
            self.calendar_window.config(bg="#F5F5DC")  # Set background color
    
            # Create the calendar widget
            self.calendar = Calendar(self.calendar_window, date_pattern='yyyy-mm-dd', font=("Arial", 12))
            self.calendar.pack(pady=20)
    
            # Button to select the date from the calendar
            select_button = tk.Button(self.calendar_window, text="Select Date", command=self.set_date_of_birth, font=("Arial", 12), bg="#6F4F37", fg="white", relief="solid", bd=2)
            select_button.pack(pady=10)
        else:
            # If the calendar window exists, bring it to the front
            self.calendar_window.lift()
            self.calendar_window.focus_force()  # Give it focus



    def set_date_of_birth(self):
        selected_date = self.calendar.get_date()
        self.selected_dob_label.config(text=f"Selected Date: {selected_date}")
        self.calendar_window.destroy()  # Close the calendar window

    def validate_user_data(self, name, dob, city, state, password, balance, contact, email, address):
        if not name or not dob or not city or not state or not password or not contact or not email or not address:
            messagebox.showerror("Input Error", "All fields are required")
            return False
        if len(password) < 6 or not re.match(r'[A-Za-z0-9@#$%^&+=]{6,}', password):
            messagebox.showerror("Password Error", "Password must be at least 6 characters and contain alphanumeric characters or special symbols.")
            return False
        if float(balance) < 2000:
            messagebox.showerror("Balance Error", "Initial balance must be at least 2000")
            return False
        if not re.match(r"^\d{10}$", contact):
            messagebox.showerror("Contact Error", "Contact number must be 10 digits")
            return False
        if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
            messagebox.showerror("Email Error", "Invalid email format")
            return False
        return True

    def generate_account_number(self):
        return ''.join(random.choices(string.digits, k=10))

    def add_user(self):
        name = self.name_entry.get()
        dob = self.selected_dob_label.cget("text").replace("Selected Date: ", "")
        city = self.city_entry.get()
        state = self.state_combobox.get()
        password = self.password_entry.get()
        balance = self.balance_entry.get()
        contact = self.contact_entry.get()
        email = self.email_entry.get()
        address = self.address_entry.get()

        if not self.validate_user_data(name, dob, city, state, password, balance, contact, email, address):
            return

        # Generate random account number
        account_number = self.generate_account_number()

        # Store user data in the database
        conn = sqlite3.connect("banking_system.db")
        cursor = conn.cursor()
        cursor.execute('''
        INSERT INTO users (name, dob, city, state, password, balance, contact, email, address, account_number)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (name, dob, city, state, password, balance, contact, email, address, account_number))
        conn.commit()

        messagebox.showinfo("Success", "User added successfully")

    def login_window(self):
        login_window = tk.Toplevel(self.root)
        login_window.title("Login")
        login_window.geometry("400x400")
        login_window.config(bg="#F5F5DC")

        login_frame = tk.Frame(login_window, bg="#F5F5DC")
        login_frame.pack(pady=20)

        self.account_number_label = tk.Label(login_frame, text="Account Number:", font=("Arial", 12), bg="#F5F5DC", fg="#6F4F37")
        self.account_number_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        self.account_number_entry = tk.Entry(login_frame, font=("Arial", 12))
        self.account_number_entry.grid(row=0, column=1)

        self.password_label = tk.Label(login_frame, text="Password:", font=("Arial", 12), bg="#F5F5DC", fg="#6F4F37")
        self.password_label.grid(row=1, column=0, padx=10, pady=10, sticky="w")
        self.password_entry = tk.Entry(login_frame, show="*", font=("Arial", 12))
        self.password_entry.grid(row=1, column=1)

        self.login_button = tk.Button(login_window, text="Login", command=self.login_user, font=("Arial", 14), bg="#6F4F37", fg="white", relief="solid", bd=2)
        self.login_button.pack(pady=20)

    def login_user(self):
        account_number = self.account_number_entry.get()
        password = self.password_entry.get()

        # Check user credentials against the database
        conn = sqlite3.connect("banking_system.db")
        cursor = conn.cursor()
        cursor.execute('''
            SELECT * FROM users WHERE account_number=? AND password=?
        ''', (account_number, password))
        result = cursor.fetchone()
        conn.close()

        if result:
            messagebox.showinfo("Login Success", "Login Successful")
        else:
            messagebox.showerror("Login Error", "Invalid Account Number or Password")

if __name__ == "__main__":
    root = tk.Tk()
    app = BankingApp(root)
    root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\dev\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\windo\AppData\Local\Temp\ipykernel_25296\3653729284.py", line 96, in show_calendar
    if not self.calendar_window or not self.calendar_window.winfo_exists():  # Check if the window exists
           ^^^^^^^^^^^^^^^^^^^^
AttributeError: 'BankingApp' object has no attribute 'calendar_window'
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\dev\anaconda3\Lib\tkinter\__init__.py", line 1968, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\windo\AppData\Local\Temp\ipykernel_25296\3653729284.py", line 96, in show_calendar
    if not self.calendar_window or not self.calendar_window.winfo_exists():  # Check if the window exists
           ^^^^^^^^^^^^^^^^^^^^
AttributeError: 'BankingApp' object has no attribute 'calendar_